In [1]:
import re
import glob

import numpy as np
import pandas as pd
import tensorflow as tf

from render import extract_params_from_path

In [6]:
def runs_to_df(paths, mode):
    data = dict(patch_size=[], filter_size=[], threshold=[], loss=[], epoch=[], step=[])
    
    for path in paths:
        params = extract_params_from_path(path)
        
        for epoch in range(31):
            steps, losses = fetch_loss(path, mode, epoch)
            
            for i in range(len(steps)):
                data['threshold'].append(params['threshold'])
                data['patch_size'].append(params['patch'])
                data['filter_size'].append(params['filter'])
                data['loss'].append(losses[i])
                data['step'].append(steps[i])
                data['epoch'].append(epoch)
        
    return pd.DataFrame(data)
    
def fetch_loss(path, mode, epoch):
    event_files = glob.glob('{}/{}-epoch{}/events.out.*'.format(path, mode, epoch))

    loss_values = []
    step_values = []
    
    if len(event_files) == 0:
        return [], []
    
    for event in tf.train.summary_iterator(event_files[0]):
        for value in event.summary.value:
            if value.tag == 'loss/loss':
                loss_values.append(value.simple_value)
                step_values.append(event.step)

    return step_values, loss_values

In [51]:
test_df = runs_to_df(glob.glob('/tmp/mrtous/patch*'), 'test')
train_df = runs_to_df(glob.glob('/tmp/mrtous/patch*'), 'train')
valid_df = runs_to_df(glob.glob('/tmp/mrtous/patch*'), 'validation')

In [57]:
def final_transform(df):
    last_step_df = df[df.groupby(['patch_size', 'filter_size', 'threshold', 'epoch'])['step'].transform(max) == df['step']]
    last_epoch_df = last_step_df[last_step_df['epoch'] == 30].drop('epoch', 1)
    last_epoch_df['total_loss'] = df.groupby(['patch_size', 'filter_size', 'threshold', 'epoch'])['loss'].transform(sum)
    last_epoch_df['total_batch'] = 128*last_epoch_df['step']
    last_epoch_df['total_pixels'] = last_epoch_df['total_batch']*last_epoch_df['patch_size']**2
    last_epoch_df['loss_per_pixel'] = last_epoch_df['total_loss']/last_epoch_df['total_pixels']
    return last_epoch_df[['patch_size', 'filter_size', 'threshold', 'loss_per_pixel']].sort_values('loss_per_pixel')

In [58]:
final_transform(test_df)

,patch_size,filter_size,threshold,loss_per_pixel
13639,15,3,20,0.001121
16615,15,6,20,0.001136
12523,15,3,10,0.001143
14011,15,3,25,0.001148
14383,15,3,5,0.001155
712,10,3,0,0.001179
3564,10,3,2,0.001190
4277,10,3,20,0.001194
2851,10,3,15,0.001197
1425,10,3,1,0.001207


In [63]:
final_transform(valid_df)

,patch_size,filter_size,threshold,loss_per_pixel
4835,5,3,5,0.002255
3905,5,3,0,0.002263
4091,5,3,1,0.002274
4277,5,3,10,0.002276
4649,5,3,2,0.002278
4463,5,3,15,0.002303
433,10,3,25,0.003764
309,10,3,2,0.003847
371,10,3,20,0.003849
61,10,3,0,0.003853
